In [1]:
from kaggle_environments import make

env = make('hungry_geese', debug=True)
foo = env.run(['submission.py', 'submission.py', 'submission.py', 'submission.py'])
env.render(mode='ipython')

Loading environment football failed: No module named 'gfootball'
0 [19.541078567504883, 19.440839767456055, 19.00003433227539, 18.22190284729004]
1 [22.0103816986084, 20.383560180664062, 14.451319694519043, 18.660064697265625]
2 [18.0863094329834, 18.455106735229492, 18.37002944946289, 16.69388198852539]
3 [17.784910202026367, 17.59716033935547, 18.899478912353516, 18.27778434753418]
0 [18.032957077026367, 19.7336483001709, -7.751813888549805, 18.6689453125]
1 [24.578611373901367, 24.103952407836914, -14.46899700164795, 20.30469512939453]
2 [18.320255279541016, 19.138893127441406, 18.554393768310547, -14.29764175415039]
3 [-9.333113670349121, 18.795961380004883, 19.55936622619629, 19.23484992980957]
0 [16.127426147460938, 17.770341873168945, 19.099533081054688, -11.583414077758789]
1 [21.12259864807129, 30.251705169677734, 9.69987678527832, 19.271987915039062]
2 [18.90093231201172, 19.119464874267578, 19.02484130859375, -15.370100021362305]
3 [-7.986437797546387, 18.691120147705078, 19